In [1]:
import findspark
findspark.init()

import requests
import json
from pprint import pp
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd

In [2]:
spark = SparkSession.builder.master("local[1]").appName('ApiData.com').getOrCreate()

In [3]:
baseurl = "https://raw.githubusercontent.com/platformps/LoanDataset/main/loan_data.json"


def main_request(baseurl):
    r = requests.get(baseurl )
    print(r.status_code)
    data = r.json()
    return data


data = main_request(baseurl)
#pp(data)

200


In [4]:
rdd= spark.sparkContext.parallelize(data)
df = rdd.toDF()
df.show()

+--------------+------------------+--------------+----------+------------+------+------+-------+-------------+-------------+
|Application_ID|Application_Status|Credit_History|Dependents|   Education|Gender|Income|Married|Property_Area|Self_Employed|
+--------------+------------------+--------------+----------+------------+------+------+-------+-------------+-------------+
|      LP001002|                 Y|             1|         0|    Graduate|  Male|medium|     No|        Urban|           No|
|      LP001003|                 N|             1|         1|    Graduate|  Male|medium|    Yes|        Rural|           No|
|      LP001005|                 Y|             1|         0|    Graduate|  Male|   low|    Yes|        Urban|          Yes|
|      LP001006|                 Y|             1|         0|Not Graduate|  Male|   low|    Yes|        Urban|           No|
|      LP001008|                 Y|             1|         0|    Graduate|  Male|medium|     No|        Urban|           No|


In [ ]:
df.write.format("jdbc").mode("append")\
        .option("url","jdbc:mysql://localhost:3306/creditcard_capstone")\
        .option("dbtable","creditcard_capstone.CDW_SAPP_loan_application")\
        .option("user","root")\
        .option("password","password")\
        .save()